In [33]:
import pandas as pd
import numpy as np
import sys

In [35]:
def scheduleMix(schedules: list[np.ndarray], group: list[int]) -> list[np.ndarray]: 
    return np.sum(schedules[group], axis = 0)

In [ ]:

class Event:
    scheduled = False
    def __init__(self, length: int, startTime : list, endTime: list, group: np.ndarray): #event length is stored in terms of how many time interval blocks it takes.
        """ 
        eventLength is stored as an int being the amount of time blocks it would take up
        start Time and end time need some work as how they will be stored is still in question
        """
        self.length = length
        self.startTime = startTime
        self.endTime = endTime
        self.group = group

    def scheduleEvent(self, schedules: np.ndarray , time: list, cancelIfEverybodyNeeded: bool):
        """Time is stored as a 2d cordinate being (day of the week, time of day by block #)"""
        if cancelIfEverybodyNeeded:
            scheduleMix(schedules, self.group)
            for idx in range(self.eventLength): # Asummes 1 means the spot is filled already
                if not schedules[idx]:
                    return
        for member in self.group:
            for idx in range(self.eventLength): 
                if not schedules[(member,idx)]:
                    schedules[(member,idx)] = 1
        self.scheduled = True


                


                

    

SyntaxError: incomplete input (1016174862.py, line 33)

In [ ]:
def bestEventTime(schedules: list[np.ndarray], event: Event):
    """ 
    generator to yield the best times in succession from a pre-computed list
    """
    mix = scheduleMix(schedules, event.group)
    
    # Create a list to store all possible time slots with their scores
    all_slots = []
    
    # Generate all possible time slots
    for i, day in enumerate(mix):
        for idx in range(len(day)-event.length):
            array = day[idx:idx+event.length]
            score = max(array) 
            all_slots.append({
                'day': i,
                'start': idx,
                'score': score
            })
    
    all_slots.sort(key=lambda x: x['score'])
    
    # Yield each slot in order
    for slot in all_slots:
        yield slot

In [ ]:
def bestEventTime(schedules: list[np.ndarray], event: Event):
    """ 
    non generator version
    """
    mix = scheduleMix(schedules, event.group)
    
    # Create a list to store all possible time slots with their scores
    all_slots = []
    
    # Generate all possible time slots
    for i, day in enumerate(mix):
        for idx in range(len(day)-event.length):
            array = day[idx:idx+event.length]
            score = max(array) 
            all_slots.append({
                'day': i,
                'start': idx,
                'score': score
            })
    
    all_slots.sort(key=lambda x: x['score'])
    
    return all_slots
  

In [42]:
def isOverlapping(range1, range2):
    start1, end1 = range1
    start2, end2 = range2

    return start1 <= end2 and start2 <= end1

isOverlapping((0,10),(6,7))

True

In [ ]:
def algowrithe(events: list[Event], schedules: list[np.ndarray], max_depth=5):
    """
    Schedule events optimally by recursively trying different arrangements
    
    Args:
        events: list of Event objects to schedule, will be sorted by group size
        schedules: list of numpy arrays representing individual availability
        max_depth: maximum recursion depth to prevent excessive processing
    
    Returns:
        tuple: (best_schedule_map, best_total_score)
    """
    events.sort(key=lambda x: len(x.group), reverse=True)
    schedule_map = np.zeros_like(schedules[0])  # Maps events to time slots
    
    def calculate_event_score(event_slots, event):
        """Calculate how many people can attend an event at given slots"""
        mix = scheduleMix(schedules, event.group)
        day, start = event_slots['day'], event_slots['start']
        array = mix[day][start:start+event.length]
        return len(event.group) * event.length - np.sum(array)  # Higher score means more can attend
    
    def is_slot_free(day, start, length):
        """Check if a time slot is available in the schedule map"""
        return not np.any(schedule_map[day][start:start+length])
    
    def schedule_event(event_idx, event, slots):
        """Schedule an event in the given slots"""
        day, start = slots['day'], slots['start']
        schedule_map[day][start:start+event.length] = event_idx + 1
    
    def unschedule_event(event_idx, event, slots):
        """Remove an event from the schedule"""
        day, start = slots['day'], slots['start']
        schedule_map[day][start:start+event.length] = 0
    
    def recursive_schedule(current_idx, current_score, depth=0):
        """
        Recursively try different event arrangements
        Returns: best_score achieved
        """
        if depth >= max_depth or current_idx >= len(events):
            return current_score
        
        current_event = events[current_idx]
        possible_slots = bestEventTime(schedules, current_event)
        best_score = current_score
        
        for slots in possible_slots:
            day, start = slots['day'], slots['start']
            
            if not is_slot_free(day, start, current_event.length):
                # Slot is occupied, try moving the conflicting event
                conflicting_event_idx = int(schedule_map[day][start]) - 1
                if conflicting_event_idx >= 0:  # Valid event index
                    conflicting_event = events[conflicting_event_idx]
                    
                    # Calculate current score for these two events
                    current_event_score = calculate_event_score(slots, current_event)
                    
                    # Temporarily move the conflicting event to try this arrangement
                    conflicting_slots = None
                    for old_slots in bestEventTime(schedules, conflicting_event):
                        if old_slots['day'] == day and old_slots['start'] == start:
                            conflicting_slots = old_slots
                            break
                    
                    if conflicting_slots:
                        old_score = calculate_event_score(conflicting_slots, conflicting_event)
                        
                        # Unschedule conflicting event
                        unschedule_event(conflicting_event_idx, conflicting_event, conflicting_slots)
                        
                        # Try scheduling current event
                        if is_slot_free(day, start, current_event.length):
                            schedule_event(current_idx, current_event, slots)
                            
                            # Recursively try to reschedule conflicting event
                            new_score = recursive_schedule(
                                conflicting_event_idx,
                                current_score - old_score + current_event_score,
                                depth + 1
                            )
                            
                            if new_score > best_score:
                                best_score = new_score
                            
                            # Undo changes if they didn't improve score
                            unschedule_event(current_idx, current_event, slots)
                        
                        # Restore conflicting event
                        schedule_event(conflicting_event_idx, conflicting_event, conflicting_slots)
            
            else:
                # Slot is free, schedule normally
                schedule_event(current_idx, current_event, slots)
                new_score = recursive_schedule(
                    current_idx + 1,
                    current_score + calculate_event_score(slots, current_event),
                    depth
                )
                
                if new_score > best_score:
                    best_score = new_score
                else:
                    # Undo if it didn't improve score
                    unschedule_event(current_idx, current_event, slots)
        
        return best_score
    
    # Start the recursive scheduling process
    final_score = recursive_schedule(0, 0)
    
    return schedule_map, final_score

In [ ]:
def algowrithe_dp(events: list[Event], schedules: list[np.ndarray]):
    """Using dynamic programming to optimize event scheduling"""
    events.sort(key=lambda x: len(x.group), reverse=True)
    schedule_map = np.zeros_like(schedules[0])
    
    # Create DP table: [event_idx][time_slot] = best_score
    dp_scores = {}  # Using dict for sparse matrix
    dp_choices = {}  # Store best choices
    
    def get_score(event_idx, available_slots):
        # Convert available slots to immutable key
        slot_key = tuple(sorted(available_slots))
        state = (event_idx, slot_key)
        
        if state in dp_scores:
            return dp_scores[state]
            
        if event_idx >= len(events):
            return 0
            
        event = events[event_idx]
        possible_slots = bestEventTime(schedules, event)
        best_score = -float('inf')
        best_choice = None
        
        for slot in possible_slots:
            if (slot['day'], slot['start']) in available_slots:
                # Calculate new available slots
                new_available = set(available_slots)
                for t in range(event.length):
                    new_available.remove((slot['day'], slot['start'] + t))
                
                score = calculate_event_score(slot, event, schedules)
                next_score = get_score(event_idx + 1, new_available)
                
                if score + next_score > best_score:
                    best_score = score + next_score
                    best_choice = slot
        
        dp_scores[state] = best_score
        dp_choices[state] = best_choice
        return best_score

    # Initialize available slots
    available_slots = set()
    for day in range(schedule_map.shape[0]):
        for time in range(schedule_map.shape[1]):
            available_slots.add((day, time))
    
    # Get optimal solution
    best_score = get_score(0, available_slots)
    
    # Reconstruct solution
    current_slots = available_slots
    for event_idx in range(len(events)):
        slot_key = tuple(sorted(current_slots))
        best_slot = dp_choices.get((event_idx, slot_key))
        if best_slot:
            schedule_event(schedule_map, event_idx, events[event_idx], best_slot)
            for t in range(events[event_idx].length):
                current_slots.remove((best_slot['day'], best_slot['start'] + t))
    
    return schedule_map, best_score

In [43]:
from pulp import *

def algowrithe_ilp(events: list[Event], schedules: list[np.ndarray]):
    """Using integer linear programming for optimal scheduling"""
    prob = LpProblem("Event_Scheduling", LpMaximize)
    
    # Decision variables
    time_slots = []
    for day in range(schedules[0].shape[0]):
        for time in range(schedules[0].shape[1]):
            time_slots.append((day, time))
    
    # Binary variables for each event and possible time slot
    choices = LpVariable.dicts("event_slot",
                             ((i, t[0], t[1]) 
                              for i in range(len(events))
                              for t in time_slots),
                             cat='Binary')
    
    # Objective function: maximize attendance
    prob += lpSum(choices[i, t[0], t[1]] * 
                 calculate_event_score({'day': t[0], 'start': t[1]}, 
                                     events[i], schedules)
                 for i in range(len(events))
                 for t in time_slots)
    
    # Constraints
    # Each event must be scheduled exactly once
    for i in range(len(events)):
        prob += lpSum(choices[i, t[0], t[1]] 
                     for t in time_slots) == 1
    
    # No overlapping events
    for t in time_slots:
        prob += lpSum(choices[i, t[0], t[1]] 
                     for i in range(len(events))) <= 1
    
    # Solve
    prob.solve()
    
    # Convert solution to schedule map
    schedule_map = np.zeros_like(schedules[0])
    for i in range(len(events)):
        for t in time_slots:
            if choices[i, t[0], t[1]].value() == 1:
                schedule_event(schedule_map, i, events[i], 
                             {'day': t[0], 'start': t[1]})
    
    return schedule_map, value(prob.objective)

ModuleNotFoundError: No module named 'pulp'

In [ ]:
def algowrithe_genetic(events: list[Event], schedules: list[np.ndarray], 
                      population_size=100, generations=50):
    """Using genetic algorithm for scheduling optimization"""
    def create_individual():
        schedule = np.zeros_like(schedules[0])
        available_slots = set((d, t) for d in range(schedule.shape[0]) 
                            for t in range(schedule.shape[1]))
        
        for idx, event in enumerate(events):
            possible = [s for s in available_slots 
                       if all((s[0], s[1] + t) in available_slots 
                             for t in range(event.length))]
            if possible:
                slot = random.choice(possible)
                for t in range(event.length):
                    available_slots.remove((slot[0], slot[1] + t))
                    schedule[slot[0]][slot[1] + t] = idx + 1
        return schedule
    
    def fitness(schedule):
        score = 0
        for idx, event in enumerate(events):
            positions = np.where(schedule == idx + 1)
            if len(positions[0]) > 0:
                day, start = positions[0][0], positions[1][0]
                score += calculate_event_score(
                    {'day': day, 'start': start}, event, schedules)
        return score
    
    # Initial population
    population = [create_individual() for _ in range(population_size)]
    
    for gen in range(generations):
        # Evaluate fitness
        fitness_scores = [fitness(ind) for ind in population]
        
        # Selection
        selected = random.choices(
            population, 
            weights=fitness_scores, 
            k=population_size)
        
        # Crossover and mutation
        new_population = []
        for i in range(0, population_size, 2):
            parent1, parent2 = selected[i], selected[i+1]
            
            # Simple crossover
            crossover_point = random.randint(0, parent1.shape[1])
            child1 = np.concatenate(
                [parent1[:,:crossover_point], 
                 parent2[:,crossover_point:]], axis=1)
            child2 = np.concatenate(
                [parent2[:,:crossover_point], 
                 parent1[:,crossover_point:]], axis=1)
            
            # Mutation
            if random.random() < 0.1:
                point = (random.randint(0, child1.shape[0]-1), 
                        random.randint(0, child1.shape[1]-1))
                child1[point] = random.randint(0, len(events))
            
            new_population.extend([child1, child2])
        
        population = new_population
    
    # Return best solution
    best_schedule = max(population, key=fitness)
    return best_schedule, fitness(best_schedule)

In [ ]:
best_schedule, total_score = algowrithe(events, schedules)

# To find where an event is scheduled:
event_idx = 0  # for the first event
event_slots = np.where(best_schedule == event_idx + 1)

In [ ]:
#My pathetic human code

def algowrithe(events: list[Event], schedules: np.ndarray):
    scheduledEventMap = np.zeros_like(schedules[0])
    timeRankings = [0 for _ in events]
    rankingIndex = [0 for _ in events]
    events.sort(key=lambda x: len(x.group), reverse=True)
    
    for event_idx, event in enumerate(events):
        timeRankings[event_idx] = bestEventTime(schedules, event)

        while True:
            time = next(timeRankings[event_idx][rankingIndex[event_idx]])
            rankingIndex[event_idx] += 1

            schedule = True
            offenders = []
            for x in range(event.length):
                if scheduledEventMap[time['day'],time['start'] + x]:
                    schedule = False
                    offenders.append(scheduledEventMap[time['day'],time['start'] + x])
            if schedule:
                for x in range(event.length):
                    scheduledEventMap[time['day'],time['start'] + x] = event_idx
                    break
                
            move_idx = offenders[0]
            for offender in offenders:
                if offender != move_idx:
                    continue
            
            while timeRankings[move_idx][rankingIndex[move_idx]+1]['score'] == timeRankings[move_idx][rankingIndex[move_idx]]:
                rankingIndex[move_idx] += 1
                if not isOverlapping()
                    pass


            
            
            

            


                    
            


        
        
    
        

    


SyntaxError: invalid syntax (3248419516.py, line 1)

In [ ]:
def simpleAlgowrithe(events: list[Event], schedules: list[np.ndarray]):
    """
    Schedule events optimally, prioritizing events with larger groups
    
    Args:
        events: list of Event objects to schedule
        schedules: list of numpy arrays representing individual schedules
    
    Returns:
        np.ndarray: Map of scheduled events where the value is the event index + 1 
                   (0 means no event scheduled)
    """
    # Initialize the schedule map with correct shape based on first schedule
    scheduledEventMap = np.zeros_like(schedules[0])
    
    # Sort events by group size in descending order (largest first)
    events.sort(key=lambda x: len(x.group), reverse=True)
    
    for event_idx, event in enumerate(events):
        # Create generator for this event's possible times
        time_generator = bestEventTime(schedules, event)
        
        scheduled = False
        for time_slot in time_generator:
            day, start = time_slot
            
            # Check if this slot is free in the scheduled event map
            slot_is_free = True
            for t in range(event.length):
                if scheduledEventMap[day][start + t] != 0:
                    slot_is_free = False
                    break
            
            if slot_is_free:
                # Schedule the event (mark with event_idx + 1 to distinguish from 0)
                for t in range(event.length):
                    scheduledEventMap[day][start + t] = event_idx + 1
                scheduled = True
                break
        
        if not scheduled:
            print(f"Warning: Could not schedule event {event_idx}")
    
    return scheduledEventMap

Experimental code below


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List
import seaborn as sns
from datetime import datetime, timedelta

def create_test_events(groups: List[np.ndarray]) -> List[Event]:
    """Create a variety of test events with different lengths and groups"""
    events = [
        Event(length=4, startTime=None, endTime=None, group=groups[0]),  # 2-hour event for first group
        Event(length=6, startTime=None, endTime=None, group=groups[1]),  # 3-hour event for everyone
        Event(length=2, startTime=None, endTime=None, group=groups[0]),  # 1-hour event for first group
        Event(length=8, startTime=None, endTime=None, group=groups[1]),  # 4-hour event for everyone
    ]
    return events

def visualize_schedule(schedule_map: np.ndarray, events: List[Event], schedules: np.ndarray):
    """Create a visual representation of the schedule"""
    plt.figure(figsize=(20, 10))
    
    # Create time labels (assuming 30-minute blocks starting at 8 AM)
    start_time = datetime.strptime("8:00", "%H:%M")
    time_labels = []
    for i in range(schedule_map.shape[1]):
        time_labels.append(start_time.strftime("%H:%M"))
        start_time += timedelta(minutes=30)
    
    # Create day labels
    day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    # Plot heatmap
    sns.heatmap(schedule_map, cmap='viridis', 
                xticklabels=time_labels[::4],  # Show every 2 hours
                yticklabels=day_labels,
                cbar_kws={'label': 'Event Number'})
    
    plt.title('Event Schedule')
    plt.xlabel('Time of Day')
    plt.ylabel('Day of Week')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def analyze_schedule(schedule_map: np.ndarray, events: List[Event], schedules: np.ndarray):
    """Analyze the schedule and print statistics"""
    print("\nSchedule Analysis:")
    print("-----------------")
    
    total_people = 0
    total_conflicts = 0
    
    for event_idx, event in enumerate(events):
        positions = np.where(schedule_map == event_idx + 1)
        if len(positions[0]) > 0:
            day, start = positions[0][0], positions[1][0]
            
            # Calculate conflicts
            mix = scheduleMix(schedules, event.group)
            conflicts = mix[day][start:start+event.length]
            
            print(f"\nEvent {event_idx + 1}:")
            print(f"  Group size: {len(event.group)} people")
            print(f"  Length: {event.length} time blocks")
            print(f"  Scheduled on: {['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'][day]}")
            print(f"  Start time: {(start * 30) // 60:02d}:{(start * 30) % 60:02d}")
            print(f"  Conflicts: {np.sum(conflicts)} person-blocks")
            print(f"  Attendance rate: {100 - (np.sum(conflicts)/(len(event.group)*event.length))*100:.1f}%")
            
            total_people += len(event.group)
            total_conflicts += np.sum(conflicts)
    
    if total_people > 0:
        print("\nOverall Statistics:")
        print(f"  Total attendance rate: {100 - (total_conflicts/total_people)*100:.1f}%")
        print(f"  Total conflicts: {total_conflicts} person-blocks")

def test_scheduling():
    """Run comprehensive tests on the scheduling algorithm"""
    # Create test events
    events = create_test_events(groups)
    
    print("Testing scheduling algorithm...")
    print(f"Number of events: {len(events)}")
    print(f"Total people: {len(np.unique(np.concatenate([event.group for event in events]))}")
    
    # Run scheduling algorithm
    schedule_map, score = algowrithe(events, exampleSchedule)
    
    # Visualize results
    visualize_schedule(schedule_map, events, exampleSchedule)
    
    # Analyze results
    analyze_schedule(schedule_map, events, exampleSchedule)
    
    # Verify schedule validity
    verify_schedule(schedule_map, events)

def verify_schedule(schedule_map: np.ndarray, events: List[Event]):
    """Verify that the schedule is valid"""
    print("\nSchedule Verification:")
    print("--------------------")
    
    # Check for overlapping events
    event_positions = {}
    for event_idx, event in enumerate(events):
        positions = np.where(schedule_map == event_idx + 1)
        if len(positions[0]) > 0:
            event_positions[event_idx] = (positions[0][0], positions[1][0], event.length)
    
    # Check for overlaps
    overlaps = []
    for event1 in event_positions:
        day1, start1, length1 = event_positions[event1]
        for event2 in event_positions:
            if event1 != event2:
                day2, start2, length2 = event_positions[event2]
                if day1 == day2:
                    if (start1 <= start2 < start1 + length1 or 
                        start2 <= start1 < start2 + length2):
                        overlaps.append((event1, event2))
    
    if overlaps:
        print("WARNING: Found overlapping events:")
        for e1, e2 in overlaps:
            print(f"  Event {e1+1} overlaps with Event {e2+1}")
    else:
        print("✓ No overlapping events found")
    
    # Check if all events are scheduled
    unscheduled = []
    for event_idx, event in enumerate(events):
        if event_idx + 1 not in schedule_map:
            unscheduled.append(event_idx)
    
    if unscheduled:
        print("WARNING: Found unscheduled events:")
        for event_idx in unscheduled:
            print(f"  Event {event_idx+1} is not scheduled")
    else:
        print("✓ All events are scheduled")

# Run the tests
if __name__ == "__main__":
    test_scheduling()

In [ ]:
# Generate example schedules as you did
exampleSchedule = np.asarray([generate_perlin_noise((7, 135), scale=5, threshold=0) for _ in range(15)])
groups = [np.asarray([5,7,9,1,11,13]), np.asarray([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])]

# Run the tests
test_scheduling()

In [28]:
q = a()
q

<generator object a at 0x0000022F79F1CA90>

In [ ]:
from perlin_noise import PerlinNoise

def generate_perlin_noise(shape, scale=10, threshold=0):
    noise = PerlinNoise(octaves=4)  
    noise_array = np.zeros(shape)
    
    x = np.linspace(0, 1, shape[0]) 
    y = np.linspace(0, 1, shape[1]) 
    xv, yv = np.meshgrid(x, y, indexing='ij')  
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            noise_value = noise([xv[i, j] * scale, yv[i, j] * scale])
            noise_array[i, j] = 1 if noise_value > threshold else 0
    
    return noise_array

exampleSchedule = np.asarray([generate_perlin_noise((7, 135), scale=5, threshold=0) for _ in range(15)])   ################## Generates 15 example schedules to use for testing

groups = [np.asarray([5,7,9,1,11,13]),np.asarray([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])] ###################### define groups

print(exampleSchedule[0])

In [24]:
scheduleMix(exampleSchedule, groups[0])

array([[0., 0., 1., 1., 1., 2., 3., 3., 3., 1., 1., 1., 1., 2., 4., 4.,
        4., 3., 3., 2., 2., 4., 4., 4., 2., 3., 4., 2., 2., 3., 3., 3.,
        2., 2., 4., 4., 2., 3., 3., 3., 2., 4., 3., 2., 2., 3., 3., 3.,
        3., 3., 3., 4., 3., 4., 2., 2., 2., 2., 2., 2., 2., 4., 5., 5.,
        5., 5., 5., 0., 1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        2., 3., 3., 3., 2., 1., 2., 2., 4., 4., 4., 4., 3., 2., 4., 4.,
        2., 2., 3., 4., 3., 3., 3., 3., 3., 2., 1., 1., 5., 5., 4., 4.,
        4., 4., 2., 2., 2., 3., 5., 4., 4., 2., 2., 2., 4., 4., 4., 4.,
        2., 2., 2., 1., 1., 0., 0.],
       [1., 2., 3., 4., 3., 3., 3., 4., 4., 3., 3., 3., 2., 3., 2., 2.,
        1., 1., 1., 0., 0., 3., 3., 2., 3., 2., 2., 1., 2., 2., 3., 3.,
        3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
        2., 2., 2., 2., 3., 4., 5., 6., 5., 3., 2., 2., 2., 1., 4., 4.,
        4., 4., 4., 4., 4., 5., 4., 3., 3., 3., 4., 4., 4., 4., 3., 3.,
        3., 4., 3., 3., 3.,